# Advanced Lane Finding Project

## Camera calibration

First I will calibrate the camera using the chessboard images. I will compute the calibration matrix and distortion coefficients

In [3]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
from pathlib import Path
import os

In [4]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

dist_parameters_sum = np.zeros([1,5])
mtx_sum = np.zeros([3,3])
dist_counter = 0

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        dist_counter = dist_counter + 1

        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        
        mtx_sum = mtx_sum + mtx
        dist_parameters_sum = dist_parameters_sum + dist

mtx_mean = mtx_sum / dist_counter
dist_parameters_mean = dist_parameters_sum / dist_counter

np.save("mtx", mtx_mean)
np.save("dist_parameters", dist_parameters_mean)

## Test camera calibration values

To test the camera calibration values the images in camera_cal were transformed with the result of the above code. And they were saved in the output_images/calibration folder.

In [5]:
# Test final values in all images
mtx = np.load("mtx.npy")
dist_parameters = np.load("dist_parameters.npy")

output_dir = "../output_images/calibration/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    # if ret == True:
    objpoints.append(objp)
    imgpoints.append(corners)

    # Draw and display the corners
    img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

    # Undistort image with previously computes parameters
    undst = cv2.undistort(img, mtx, dist_parameters, None, mtx)

    cv2.imshow('Original', img)
    cv2.imshow('Undistort', undst)
    cv2.waitKey(500)
    
    cv2.imwrite(output_dir + os.path.basename(fname), undst)
        
cv2.destroyAllWindows()

The resulted images show a proper undistortion as we can see:

Before:

![Lanes Image](../camera_cal/calibration1.jpg) 

Alfer

![Lanes Image](../output_images/calibration/calibration1.jpg) 